In [50]:

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Get weights

Proposed API

```python
ResNet.resnet50(pretrained=True, dataset='imagenet')

ResNet.pretrained_models
```



In [56]:
timm.list_models('se**')

['selecsls42',
 'selecsls42b',
 'selecsls60',
 'selecsls60b',
 'selecsls84',
 'semnasnet_050',
 'semnasnet_075',
 'semnasnet_100',
 'semnasnet_140',
 'senet154',
 'seresnet18',
 'seresnet34',
 'seresnet50',
 'seresnet50tn',
 'seresnet101',
 'seresnet152',
 'seresnext26_32x4d',
 'seresnext26d_32x4d',
 'seresnext26t_32x4d',
 'seresnext26tn_32x4d',
 'seresnext50_32x4d',
 'seresnext101_32x4d',
 'seresnext101_32x8d']

In [59]:
import timm
m = timm.create_model('seresnet101', pretrained=True)
m

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (se): SEModule(
        (avg_pool): AdaptiveAvgPool2d

In [73]:
from dataclasses import dataclass
from typing import Tuple, List
from pprint import pprint
from benchmark import benchmark
from glasses.utils.ModuleTransfer import ModuleTransfer
from functools import partial
from glasses.nn.models import *
from torchvision.models import resnet18, resnet34, resnet50, resnet101, resnet152, resnext50_32x4d, resnext101_32x8d, wide_resnet50_2, wide_resnet101_2
from tqdm.autonotebook import tqdm
import torchvision.transforms as T
import torch.nn as nn
import torch
from PIL import Image
from glasses.utils.PretrainedWeightsProvider import Config


models_weights = {
    'seresnet50': Config(),
#     'efficientnet_b4': Config(resize=380, input_size=380, interpolation='bicubic')
}

zoo_models_mapping = {
    'seresnet50': [partial(timm.create_model, 'seresnet101',pretrained=True), SEResNet.cse_resnet34],
   
#   'efficientnet_b4': [partial(timm.create_model, 'tf_efficientnet_b4', pretrained=True), EfficientNet.efficientnet_b4]
}



# pprint(models_weights)

In [74]:
def clone_model(src: nn.Module, dst: nn.Module) -> nn.Module:
    src = src.eval()
    dst = dst.eval()

    x = torch.rand((1, 3, 224, 224))
    a = src(x)
    b = dst(x)

    # assert not torch.equal(a, b)

    ModuleTransfer(src, dst)(x)

    # a = src(x)
    # b = dst(x)

    # assert torch.equal(a, b)

    return dst


In [75]:
bar = tqdm(zoo_models_mapping.items())

for name, mapping in bar:
    
    src, dst = mapping[0](), mapping[1]()
    tr = models_weights[name].transform
    print(tr)
    clone_model(src, dst)
    bar.set_description(name)
    res = benchmark(dst.cuda(), tr)
    print(name, res)
    
    res = benchmark(src.cuda(), tr)
    print(name, res)

Compose(
    Resize(size=256, interpolation=PIL.Image.BILINEAR)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
)



Exception: Numbers of operations are different. Source module has 275 operations while destination module has 105.

In [ ]:
[{'original_time': 84.83511066436768,████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌| 49866/50000 [00:05<00:00, 9220.38it/s]
  'original_top1': 0.74654,
  'original_top5': 0.91924},
 {'original_time': 122.87314462661743,
  'original_top1': 0.74602,
  'original_top5': 0.9159},
 {'original_time': 168.03247451782227,
  'original_top1': 0.79398,
  'original_top5': 0.94576},
 {'original_time': 281.6872932910919,
  'original_top1': 0.81218,
  'original_top5': 0.95528},
 {'original_time': 588.1660254001617,
  'original_top1': 0.8279,
  'original_top5': 0.96234},
 {'original_time': 1126.7628757953644,
  'original_top1': 0.8346,
  'original_top5': 0.96624}]

In [ ]:
EfficientNet.b0()

In [ ]:
from efficientnet_pytorch import EfficientNet as EfficientNetPytorch
from glasses.nn.models import  EfficientNet
from torchsummary import summary



summary(EfficientNet.b0().cuda(), (3, 224, 224))

In [ ]:
model = EfficientNetPytorch.from_pretrained('efficientnet-b0')
model.set_swish(False)
summary(model.cuda(), (3, 224, 224))

In [ ]:
model